In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import classification_report, confusion_matrix
from datetime import datetime
from sklearn.ensemble import RandomForestRegressor
import matplotlib.pyplot as plt
pd.plotting.register_matplotlib_converters()
%matplotlib inline
import seaborn as sns

from sklearn.metrics import mean_squared_error
from catboost import CatBoostRegressor
from sklearn.model_selection import StratifiedKFold
from xgboost import XGBRegressor
from xgboost import XGBClassifier
from sklearn.metrics import mean_absolute_error
from sklearn import metrics
from scipy.stats import rankdata
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import normalize
from sklearn.feature_selection import mutual_info_regression
import numpy as np
from sklearn.model_selection import TimeSeriesSplit
from sklearn.multioutput import MultiOutputRegressor
from statsmodels.tsa.seasonal import seasonal_decompose
from statsmodels.tsa.stattools import adfuller
from statsmodels.graphics.tsaplots import plot_acf
import eli5
from eli5.sklearn import PermutationImportance
from scipy import stats
#from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
#from sklearn.model_selection import StratifiedKFold
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_curve

print("finished")

In [ ]:
trainlfilepath="../input/tabular-playground-series-apr-2022/train_labels.csv"
dftrainl=pd.read_csv(trainlfilepath)

trainfilepath="../input/tabular-playground-series-apr-2022/train.csv"
dftrain=pd.read_csv(trainfilepath)

testfilepath="../input/tabular-playground-series-apr-2022/test.csv"
dftest=pd.read_csv(testfilepath)

subfilepath="../input/tabular-playground-series-apr-2022/sample_submission.csv"
dfsub=pd.read_csv(subfilepath)


In [ ]:
#merge labels to train data frame
df=dftrain.merge(dftrainl, on="sequence",how="left")
df

In [ ]:
#eda conclusion:
#target distributed almost qeualy between 0s and 1s
#there's no NA values
#subject distributed evenly between states
#no found pattern is distribution of states between subjects
#sensors have no correlation between them
#all sensors has a lot of outliers
#all sensors without outliers are in [-2:2]
#target and features have no correlation


Evaluate algorithms

In [ ]:
#train-validation split function
#better to mimic test. As seen of now test is sequences which continue train
#so train valid split is done next way: 30% (8k) of tail are validation
def trainvalidsplit(x):
    xtrain=x[x.sequence<=18000]
    xvalid=x[x.sequence>18000]
    ytrain=xtrain.pop("state")
    yvalid=xvalid.pop("state")
    return xtrain,xvalid,ytrain,yvalid

In [ ]:
xtrain,xvalid,ytrain,yvalid=trainvalidsplit(df)
print(xtrain.shape)
print(ytrain.shape)
print(xvalid.shape)
print(yvalid.shape)

In [ ]:
#validation function AUC
def aucvalue(yvalid,predictions):
    aucvalue= metrics.roc_auc_score( y_true=yvalid,y_score=predictions)
    return aucvalue

In [ ]:
#combine  train and test to prepare semultaniously
dffull=pd.concat([df,dftest])

Feature enginering and data clean

In [ ]:
#no need to work with NA couse no NA values
#require reduce number of data

In [ ]:
#will add mean, std,min max 25 50 75
dfgroupped=dffull.groupby(dffull.sequence, as_index=False).agg({'subject':'last','sensor_00':['std','median'],'sensor_01':['std','median'],'sensor_02':['std','median'],'sensor_03':['std','median'],'sensor_04':['std','median'],'sensor_05':['std','median'],'sensor_06':['std','median'],'sensor_07':['std','median'],'sensor_08':['std','median'],'sensor_09':['std','median'],'sensor_10':['std','median'],'sensor_11':['std','median'],'sensor_12':['std','median']})
dfgroupped

In [ ]:
dfgroupped.columns=['sequence','subject','s00std','s00median','s01std','s01median','s02std','s02median','s03std','s03median','s04std','s04median','s05std','s05median','s06std','s06median','s07std','s07median','s08std','s08median','s09std','s09median','s10std','s10median','s11std','s11median','s12std','s12median']

dfgroupped

In [ ]:
#get sequence values for train and test data set to split data frame
trainseqlist=df.sequence.to_list()
trainseqlist=set(trainseqlist)
trainseqlist=list(trainseqlist)

testseqlist=dftest.sequence.to_list()
testseqlist=set(testseqlist)
testseqlist=list(testseqlist)

In [ ]:
dftrain=dfgroupped.loc[dfgroupped.sequence.isin(trainseqlist),:]
dftrain=dftrain.merge(dftrainl, on="sequence",how="left")
dftrain

In [ ]:
dftest=dfgroupped.loc[dfgroupped.sequence.isin(testseqlist),:]
dftest

In [ ]:
xtrain,xvalid,ytrain,yvalid=trainvalidsplit(dftrain)
print(xtrain.shape)
print(ytrain.shape)
print(xvalid.shape)
print(yvalid.shape)

In [ ]:
from tensorflow import keras
from tensorflow.keras import layers

model = keras.Sequential([
    layers.Dense(4, activation='relu', input_shape=[28]),
    layers.Dense(4, activation='relu'),   
    layers.Dense(4, activation='relu'),   
    layers.Dense(4, activation='relu'),   
    layers.Dense(1, activation='sigmoid'),
])

In [ ]:
model.compile(
    optimizer='adam',
    loss='binary_crossentropy',
    metrics=['binary_accuracy'],
)

In [ ]:
early_stopping = keras.callbacks.EarlyStopping(
    patience=10,
    min_delta=0.001,
    restore_best_weights=True,
)

history = model.fit(
    xtrain, ytrain,
    validation_data=(xvalid, yvalid),
    batch_size=512,
    epochs=1000,
    callbacks=[early_stopping],
    verbose=1, # hide the output because we have so many epochs
)

In [ ]:
# convert the training history to a dataframe
history_df = pd.DataFrame(history.history)
# use Pandas native plot method
history_df['loss'].plot();

train model on full list of columns and full train set

In [ ]:
dftrain

In [ ]:
ydftrain=dftrain.pop('state')

In [ ]:
predictproba=model.predict(dftest)
predictions = [np.round(value) for value in predictproba]
pred=pd.DataFrame(predictions, index=dftest.index)


In [ ]:
output=pd.DataFrame({"sequence":dftest.sequence,"state":pred[0]})
output.to_csv("submission7.csv",index=False)

In [ ]:
output